<a href="https://colab.research.google.com/github/fabiobasson/Bi-Master/blob/main/geological_image_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from os import getcwd
import zipfile
import shutil
import numpy as np
import glob
import random
import pandas as pd
import matplotlib.pyplot as plt


from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
!rm -rf geological_similarity

**Importar arquivos de configuração do Kaglle**

In [43]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"fabiobasson","key":"d71cbfdfd23379909173204eac74cf96"}'}

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

###  Download dos dados / descompactando os dados

In [49]:
!kaggle datasets download tanyadayanand/geological-image-similarity
! unzip -qq  geological-image-similarity

geological-image-similarity.zip: Skipping, found more recently modified local copy (use --force to force download)


In [33]:
!kaggle datasets download fabiobasson/geological-similarity
! unzip -qq  geological-similarity

geological-similarity.zip: Skipping, found more recently modified local copy (use --force to force download)


In [50]:
! rm -rf gneiss andesite

### Data Preprocessing

In [53]:
andesite_dir = glob.glob('geological_similarity/andesite/*.jpg')
gneiss_dir = glob.glob('geological_similarity/gneiss/*.jpg')
marble_dir = glob.glob('geological_similarity/marble/*.jpg');quartzite_dir = glob.glob('geological_similarity/quartzite/*.jpg')
rhyolite_dir = glob.glob('geological_similarity/rhyolite/*.jpg');schist_dir = glob.glob('geological_similarity/schist/*.jpg')
print(len(andesite_dir));print(len(gneiss_dir)); print(len(marble_dir)); print(len(quartzite_dir));  print(len(schist_dir))


5000
5000
4998
5000
5000


In [54]:
andesite_dir[0:4]

['geological_similarity/andesite/9FUK8.jpg',
 'geological_similarity/andesite/20XR8.jpg',
 'geological_similarity/andesite/PT0WZ.jpg',
 'geological_similarity/andesite/Z7KXW.jpg']

In [ ]:
#geological_similarity = 'geological_similarity'
#subfolders = [f.path for f in os.scandir(geological_similarity) if f.is_dir()]


In [55]:
class_names = ['andesite', 'gneiss', 'marbler', 'quartzite', 'rhyolite',
               'schist']
class_names[0]

'andesite'

In [56]:
gneiss_df=[]
andesite_df=[]
for i in andesite_dir:
    andesite_df.append([i,class_names[0]])
for j in gneiss_dir:
   gneiss_df.append([j,class_names[1]])
df = andesite_df + gneiss_df
random.shuffle(df)
df

[['geological_similarity/gneiss/UKTLF.jpg', 'gneiss'],
 ['geological_similarity/andesite/7O3A3.jpg', 'andesite'],
 ['geological_similarity/gneiss/HZ1WR.jpg', 'gneiss'],
 ['geological_similarity/gneiss/5EIEM.jpg', 'gneiss'],
 ['geological_similarity/gneiss/SMU1Q.jpg', 'gneiss'],
 ['geological_similarity/andesite/4ES85.jpg', 'andesite'],
 ['geological_similarity/gneiss/NO2DE.jpg', 'gneiss'],
 ['geological_similarity/andesite/O10LI.jpg', 'andesite'],
 ['geological_similarity/andesite/4CQ3S.jpg', 'andesite'],
 ['geological_similarity/gneiss/GCULR.jpg', 'gneiss'],
 ['geological_similarity/andesite/K5LBC.jpg', 'andesite'],
 ['geological_similarity/andesite/V9DDM.jpg', 'andesite'],
 ['geological_similarity/andesite/P4TEW.jpg', 'andesite'],
 ['geological_similarity/gneiss/5ULQD.jpg', 'gneiss'],
 ['geological_similarity/gneiss/SYK4W.jpg', 'gneiss'],
 ['geological_similarity/andesite/FLIHH.jpg', 'andesite'],
 ['geological_similarity/andesite/9EAE9.jpg', 'andesite'],
 ['geological_similarity/gnei

In [57]:
data_df = pd.DataFrame(df,columns=['path','label'])
data_df

,path,label
0,geological_similarity/gneiss/UKTLF.jpg,gneiss
1,geological_similarity/andesite/7O3A3.jpg,andesite
2,geological_similarity/gneiss/HZ1WR.jpg,gneiss
3,geological_similarity/gneiss/5EIEM.jpg,gneiss
4,geological_similarity/gneiss/SMU1Q.jpg,gneiss
...,...,...
9995,geological_similarity/gneiss/I1TJ9.jpg,gneiss
9996,geological_similarity/andesite/1RJ9W.jpg,andesite
9997,geological_similarity/andesite/PMR5O.jpg,andesite
9998,geological_similarity/gneiss/IT146.jpg,gneiss


In [79]:
data_df.columns

RangeIndex(start=0, stop=18, step=1)